<a href="https://colab.research.google.com/github/hqy5038/ist-303-xr-team-ex-2/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import sys
# append the function .py file file path to the system path
# function.py is uploaded to /content/drive/MyDrive/Functions folder
if '/content/drive/MyDrive/KDDM2024/Functions' not in sys.path:
 sys.path.append('/content/drive/MyDrive/KDDM2024/Functions')

In [11]:
import function as fun

In [1]:
import pandas as pd

credit_rating_data = pd.read_excel('/content/drive/MyDrive/KDDMFINAL/CreditRating Data 2012 (1).xlsx')
family_income = pd.read_excel('/content/drive/MyDrive/KDDMFINAL/Family_Income.xlsx')
HS_Academics = pd.read_excel('/content/drive/MyDrive/KDDMFINAL/HS_Academics 2012.xlsx')
Loanamount = pd.read_excel('/content/drive/MyDrive/KDDMFINAL/LoanAmount.xlsx')
Parent_Edu = pd.read_excel('/content/drive/MyDrive/KDDMFINAL/Parents_Education.xlsx')
Characteristics = pd.read_excel('/content/drive/MyDrive/KDDMFINAL/Personality Characteristics Data.xlsx')
Region_Marital = pd.read_excel('/content/drive/MyDrive/KDDMFINAL/Region, Marital Status.xlsx')
no_payment = pd.read_csv('/content/drive/MyDrive/KDDMFINAL/Updated_no_payment_due.csv')
filed_for_bankruptcy = pd.read_csv('/content/drive/MyDrive/KDDMFINAL/updated_bankrupcy.csv')
disabled = pd.read_csv('/content/drive/MyDrive/KDDMFINAL/updated_disabled.csv')
enlist = pd.read_csv('/content/drive/MyDrive/KDDMFINAL/updated_enlist.csv')
enrolled = pd.read_csv('/content/drive/MyDrive/KDDMFINAL/updated_enrolled.csv')
longest_absence = pd.read_csv('/content/drive/MyDrive/KDDMFINAL/updated_longest_absence.csv')
male_data = pd.read_csv('/content/drive/MyDrive/KDDMFINAL/updated_male.csv')
unemployed = pd.read_csv('/content/drive/MyDrive/KDDMFINAL/updated_unemployed.csv')

#Binary Encoding

In [2]:
disabled.replace('disabled', 1, inplace=True)
unemployed.replace('unemployed', 1, inplace=True)
male_data.replace('male', 1, inplace=True)

#Merge the data

In [20]:
df = credit_rating_data.merge(family_income, on='STUDENT_ID', how='left')
df = df.merge(HS_Academics, on='STUDENT_ID', how='left')
df = df.merge(Loanamount, on='STUDENT_ID', how='left')
df = df.merge(Parent_Edu, on='STUDENT_ID', how='left')
df = df.merge(Characteristics, on='STUDENT_ID', how='left')
df = df.merge(Region_Marital, on='STUDENT_ID', how='left')
df = df.merge(no_payment, on='STUDENT_ID', how='left')
df = df.merge(filed_for_bankruptcy, on='STUDENT_ID', how='left')
df = df.merge(disabled, on='STUDENT_ID', how='left')



enrolled['School'] = enrolled['School'].astype(str)
# Group by 'STUDENT_ID' and combine 'School' entries
df_combined = enrolled.groupby('STUDENT_ID').agg({
    'School': ' & '.join
}).reset_index()

# Add a new column that counts the number of schools for each student
df_combined['Num_Schools'] = df_combined['School'].apply(lambda x: len(set(x.split(' & '))))

#Add 'Enrolled' column
df_combined['Enrolled'] = 1
df = df.merge(df_combined, on='STUDENT_ID', how='left')

# Group by 'STUDENT_ID' and combine 'Service' entries
enlist['Service'] = enlist['Service'].astype(str)

df_combined = enlist.groupby('STUDENT_ID').agg({
    'Service': ' & '.join
}).reset_index()

#Add 'Enlisted' column
df_combined['Enlisted'] = 1

df = df.merge(df_combined, on='STUDENT_ID', how='left')
df = df.merge(longest_absence, on='STUDENT_ID', how='left')
df = df.merge(unemployed, on='STUDENT_ID', how='left')
df = df.merge(male_data, on='STUDENT_ID', how='left')

#Fill the null value
df['Status'] = df['Status'].fillna(0)
df['Employment status']= df['Employment status'].fillna(0)
df['Gender'] = df['Gender'].fillna(0)
df['Enlisted'] = df['Enlisted'].fillna(0)
df['Enrolled'] = df['Enrolled'].fillna(0)
df['Num_Schools'] = df['Num_Schools'].fillna(0)

##We found missing values  in 'HS_English'column and decided to replace the missing values with the mode value
mode_HS_English = df['HS_English'].replace('', pd.NA).dropna().mode()[0]

# Replace NaN and empty strings with the mode value
df['HS_English'] = df['HS_English'].replace('', mode_HS_English)
df['HS_English'].fillna(mode_HS_English, inplace=True)

df['HS_English'] = df['HS_English'].astype(int)

print(df)


     STUDENT_ID  Credit_Score_Raw  Credit_Score_AgeAdj  Family_Income  \
0             1          0.495227             0.767967      109770.95   
1             2          0.987649             0.529815       58442.99   
2             3          0.249315             0.589539       97125.87   
3             4          0.653183             0.805702       55256.40   
4             5          0.273099             0.415433       58187.16   
..          ...               ...                  ...            ...   
995         996          0.612762             0.780557       83784.09   
996         997          0.150267             0.455959       70869.55   
997         998          0.777285             0.441015       67495.76   
998         999          0.989682             0.597839       65238.16   
999        1000          0.639912             0.808111       54206.48   

     HS_Math  HS_Science  HS_English  Loan_Amount PARENTS_EDUCATION_LEVEL  \
0         10          10          10        19

#Manage Unique values

In [21]:
# Normalize and replace variations of 'HS or Lower' with 'HS or Lower'
df['PARENTS_EDUCATION_LEVEL'] = df['PARENTS_EDUCATION_LEVEL'].replace({
    ' VERY HS or Lower': 'HS or Lower',  # Note the leading space
    'GREAT': 'HS or Lower',
    'VERY HS or Lower': 'HS or Lower'   # Handling potential typo or format issues
})

# Verify the changes
print(df['PARENTS_EDUCATION_LEVEL'].unique())


["Bachelor's" 'Doctorate' 'HS or Lower' "Master's"]


In [22]:
df['Married'] = df['Married'].replace(2, 1)

# Check unique values again to confirm changes
print(df['Married'].unique())


[1 0]


In [23]:
df['Country'] = df['Country'].replace({
    'Belize': 'USA'
})
print(df['Country'].unique())

['USA']


In [24]:
print(df.columns)

Index(['STUDENT_ID', 'Credit_Score_Raw', 'Credit_Score_AgeAdj',
       'Family_Income', 'HS_Math', 'HS_Science', 'HS_English', 'Loan_Amount',
       'PARENTS_EDUCATION_LEVEL', 'LongTermPlannerScore ', 'DecisionStyle',
       'PowerOrientedScore', 'CommunityOrientedScore', 'Country', 'Region',
       'Married', 'NoPaymentDue', 'bankruptcy Status', 'Status', 'School',
       'Num_Schools', 'Enrolled', 'Service', 'Enlisted',
       'longest absense from school in months', 'Employment status', 'Gender'],
      dtype='object')


#Manage Column name

In [25]:
df.rename(columns={'longest absense from school in months': 'Longest_Absence_From_School'}, inplace=True)
df.rename(columns={'STUDENT_ID': 'StudentID'}, inplace=True)
df.rename(columns={'Status': 'Disabled'}, inplace=True)
df.rename(columns={'Employment status': 'Unemployed'}, inplace=True)
df.rename(columns={'Married': 'Marital_Status'}, inplace=True)
df.rename(columns={'bankruptcy Status': 'Filed_for_Bankrupcy'}, inplace=True)
df.rename(columns={'LongTermPlannerScore': 'LongTermPlanningScore'}, inplace=True)
df.rename(columns={'PARENTS_EDUCATION_LEVEL': 'Parents’ Education Level'}, inplace=True)
df.rename(columns={'Credit_Score_Raw': 'Credit Score_Raw'}, inplace=True)
df.rename(columns={'Credit_Score_AgeAdj': 'Credit Score_AgeAdj'}, inplace=True)
df.rename(columns={'Loan_Amount': 'Loan Amount'}, inplace=True)

In [26]:
fun.metadata(df)

##1. HS_English missing value? ---->mode
#3. Married has three unique values? ---->change 2 to 1
##4. is normal to have a credit score raw = 1?

,column_name,datatype,missing_percent,unique
0,StudentID,int64,0.0,1000
1,Credit Score_Raw,float64,0.0,996
2,Credit Score_AgeAdj,float64,0.0,1000
3,Family_Income,float64,0.0,1000
4,HS_Math,int64,0.0,9
5,HS_Science,int64,0.0,8
6,HS_English,int64,0.0,9
7,Loan Amount,int64,0.0,912
8,Parents’ Education Level,object,0.0,4
9,LongTermPlannerScore,float64,0.0,1000


In [30]:
# Save the merged data to an Excel file
df.to_excel('/content/drive/MyDrive/KDDMFINAL/Corrected_Merged_Data.xlsx', index=False)

In [28]:
df.dtypes

StudentID                        int64
Credit Score_Raw               float64
Credit Score_AgeAdj            float64
Family_Income                  float64
HS_Math                          int64
HS_Science                       int64
HS_English                       int64
Loan Amount                      int64
Parents’ Education Level        object
LongTermPlannerScore           float64
DecisionStyle                   object
PowerOrientedScore             float64
CommunityOrientedScore         float64
Country                         object
Region                           int64
Marital_Status                   int64
NoPaymentDue                    object
Filed_for_Bankrupcy             object
Disabled                       float64
School                          object
Num_Schools                    float64
Enrolled                       float64
Service                         object
Enlisted                       float64
Longest_Absence_From_School      int64
Unemployed               